# Mask R-CNN Beginner's Tutorial


## Install Google drive


In [ ]:
from google.colab import drive
import sys, datetime

# Connect Google drive with authentication
drive.mount("/content/gdrive", force_remount=True)

# Set root dir of the project
ROOT_DIR = "/content/gdrive/MyDrive/MaskRCNN_Tutorial/"
sys.path.insert(0, ROOT_DIR)
# List file information in the path
%ls $ROOT_DIR

# Set training phase with time stamp
train_phase = "OralCancer_{:%Y%m%dT%H%M}".format(datetime.datetime.now())
print("\nSet training phase: {}".format(train_phase))

Mounted at /content/gdrive
DATASET/  dataset_split.py  MaskRCNN_Tutorial.ipynb  mrcnn/  README.md

Set training phase: OralCancer_20220618T2010


## Import required library

In [ ]:
# Import library
import os
import json
import skimage.draw
import numpy as np
from google.colab.patches import cv2_imshow

# Import Mask RCNN
from mrcnn import utils
from mrcnn import visualize
from mrcnn.model import log
from mrcnn.config import Config
from mrcnn import model as modellib

## Configurations

Set model configuration by the ModelConfig class.

In [ ]:
class ModelConfig(Config):
    """Configuration for training on the oral dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "OralCancer"

    # Set specific dataset dir
    DATASET_DIR = ROOT_DIR + "DATASET/DATA_220617"
    
    # Number of training epochs
    EPOCH = 10
    
    # Number of training steps per epoch
    STEPS_PER_EPOCH = 10

    # Weight
    WEIGHT = "coco"
    # [Option]: "coco", "imagenet", "last", "<trained>"
    
    # Directory to save logs and model checkpoints
    LOGS_DIR = ROOT_DIR + "LOG"
    
    # Batch size
    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 3

    # Number of classes (including background)
    NUM_CLASSES = 1 + 3  # Background + number of classes

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9


## Dataset

In [ ]:
class OralDataset(utils.Dataset):

    def load_dataset(self, dataset_dir, subset):
        """ Load a subset of the oral dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes according to the numbe of classes required to detect
        self.add_class("OralCancer", 1, "G")
        self.add_class("OralCancer", 2, "Y")
        self.add_class("OralCancer", 3, "R")
        print("\n[INFO] self.class_info: \n{}".format(self.class_info))
        
        # Train or validation dataset?
        assert subset in ["train", "val", "test"]
        dataset_dir = os.path.join(dataset_dir, subset)
        
        # Load annotations
        # VGG Image Annotator (up to version 1.6) saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        
        # Note: In VIA 2.0, regions was changed from a dict to a list.
        annotations = json.load(open(os.path.join(dataset_dir, subset+".json")))
        annotations = list(annotations.values())  # Remove the dict keys
        
        # The VIA tool saves images in the JSON even if they don't have any annotations.
        # Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
        
        # Add images
        for a in annotations:
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. These are stores in the
            # shape_attributes (see json format above)
            # The if condition is needed to support VIA versions 1.x and 2.x.
            if type(a['regions']) is dict:
                polygons = [r['shape_attributes'] for r in a['regions'].values()]
            else:
                polygons = [r['shape_attributes'] for r in a['regions']]
            
            # Read labeled class name
            label_cls = [r['region_attributes']['OralCancer'] for r in a['regions']]
            # Encode class name by class ID
            class_ids = {'G': 1,
                    'Y': 2,
                    'R': 3}
            label_ids = [class_ids[a] for a in label_cls]
            
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]
            
            self.add_image(
                "OralCancer",
                image_id=a['filename'], # use file name as a unique image id
                class_ids=label_ids,
                path=image_path,
                width=width,height=height,
                polygons=polygons
            )
    
    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a oral dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "OralCancer":
            return super(self.__class__, self).load_mask(image_id)
        
        class_ids = image_info["class_ids"]
        class_ids = np.array(class_ids, dtype=np.int32)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1
        
        return mask.astype(np.bool_), class_ids

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "OralCancer":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [ ]:
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = OralDataset()
    dataset_train.load_dataset(ROOT_DIR+config.DATASET_DIR, "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = OralDataset()
    dataset_val.load_dataset(ROOT_DIR+config.DATASET_DIR, "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
          learning_rate=config.LEARNING_RATE,
          epochs=config.EPOCH,
          layers='heads'
    )

## Train on the dataset

In [ ]:
if __name__ == '__main__':
    
    # Configurations
    config = ModelConfig()
    config.display()

    # Create model
    model = modellib.MaskRCNN(mode="training", config=config, model_dir=config.LOGS_DIR)

    # Select weights file to load
    if config.WEIGHT.lower() == "coco":
        weights_path = "mask_rcnn_coco.h5" #COCO_WEIGHTS_PATH
        # Download weights file
        if not os.path.exists(weights_path):
            utils.download_trained_weights(weights_path)
    elif config.WEIGHT.lower() == "imagenet":
        # Start from ImageNet trained weights
        weights_path = model.get_imagenet_weights()
    elif config.WEIGHT.lower() == "last":
        # Find last trained weights
        weights_path = model.find_last()
    else:
        weights_path = config.WEIGHT
    
    # Load weights
    print("Loading weights ", weights_path)
    if config.WEIGHT.lower() == "coco":
        # Exclude the last layers because they require a matching
        # number of classes
        model.load_weights(weights_path, by_name=True, exclude=[
                  "mrcnn_class_logits", "mrcnn_bbox_fc",
                  "mrcnn_bbox", "mrcnn_mask"])
    else:
      model.load_weights(weights_path, by_name=True)
    
    # Train model
    train(model)


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     3
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DATASET_DIR                    DATASET/DATA_220617
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
EPOCH                          10
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 3
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                16
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOGS_DIR                       LOG
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn

IndexError: ignored

## Exhibit training curves in tensorboard:

In [ ]:
%load_ext tensorboard
%tensorboard --logdir config.LOG_DIR

In [1]:
"""
from sklearn.metrics import confusion_matrix
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage.io

MODEL_PATH = os.path.join(config.LOG_DIR, "./oral20220624T1332/mask_rcnn_OralCancer_0036.h5") #change

#config.OUTPUT_DIR = "./OUTPUT/oral20220624T1332_epoch0036/"
os.makedirs(config.OUTPUT_DIR, exist_ok=True)

# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    #GPU_COUNT = 1
    IMAGES_PER_GPU = 1
config = InferenceConfig()
config.display()

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
TEST_MODE = "inference"

def get_ax(rows=1, cols=1, size=8):
    #Return a Matplotlib Axes array to be used in
    #all visualizations in the notebook. Provide a
    #central point to control graph sizes.
    #Adjust the size attribute to control how big to render images
    
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

# Test dataset
dataset_test = OralDataset()
dataset_test.load_dataset(config.DATASET_DIR, "test")
dataset_test.prepare()

model = modellib.MaskRCNN(mode="inference", config=config, model_dir=config.LOG_DIR)

# Set weights file path
if MODEL_PATH is not None:
    weights_path = MODEL_PATH
else:
    weights_path = model.find_last()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

# ## Run Detection

# Initial ground-truth and predictions lists
gt_tot_per_image = np.array([])
pred_tot_per_image = np.array([])
gt_tot = np.array([])
pred_tot = np.array([])

# Initial mAP list
mAP_ = []
mprecision_ = []
mrecall_ = []
mdice = []
class_names = []
class_names_index = []
APs = []
count1 = 0

"""

'\nfrom sklearn.metrics import confusion_matrix\nimport matplotlib\nimport matplotlib.pyplot as plt\nimport matplotlib.patches as patches\nimport skimage.io\n\nMODEL_PATH = os.path.join(config.LOG_DIR, "./oral20220624T1332/mask_rcnn_OralCancer_0036.h5") #change\nOUTPUT_DIR = "./RESULT/oral20220419T0338_epoch0036/"\nos.makedirs(OUTPUT_DIR, exist_ok=True)\n\n# Override the training configurations with a few\n# changes for inferencing.\nclass InferenceConfig(config.__class__):\n    # Run detection on one image at a time\n    GPU_COUNT = 1\n    IMAGES_PER_GPU = 1\nconfig = InferenceConfig()\nconfig.display()\n\n# Inspect the model in training or inference modes\n# values: \'inference\' or \'training\'\n# TODO: code for \'training\' test mode not ready yet\nTEST_MODE = "inference"\n\ndef get_ax(rows=1, cols=1, size=8):\n    #Return a Matplotlib Axes array to be used in\n    #all visualizations in the notebook. Provide a\n    #central point to control graph sizes.\n    #Adjust the size attri

In [ ]:
"""
# image_id = random.choice(dataset.image_ids, 32) # Random choose several testing samples
for image_id in dataset.image_ids:
    image, image_meta, gt_class_id, gt_bbox, gt_mask = modellib.load_image_gt(dataset, config, image_id) #change, use_mini_mask=False
    info = dataset.image_info[image_id]
    print("\nimage ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, dataset.image_reference(image_id)))
    
    # molded_images = np.extend_dims(modellib.mold_image(image, config), 0) #err
    molded_image = np.expand_dims(image, 0)
    
    # Run object detection
    # results = model.detect([image], verbose=1)
    results = model.detect(molded_image, verbose=0)

    # Display results
    ax = get_ax() #1
    r = results[0]
    
    # - Concatenate all materials for "confusion matrix" - 
    if count1 == 0:
        # Concatenate "ground truth" class, bbox, mask
        concat_gt_class_id, concat_gt_bbox, concat_gt_mask = gt_class_id, gt_bbox, gt_mask
        # Concatenate all model "detection result"
        concat_r_rois, concat_r_masks, concat_r_class_ids, concat_r_scores = r['rois'], r['masks'], r['class_ids'], r['scores']
    else:
        # Concatenate "ground truth" class, bbox, mask
        concat_gt_class_id = np.concatenate((concat_gt_class_id, gt_class_id), axis=0)
        concat_gt_bbox = np.concatenate((concat_gt_bbox, gt_bbox), axis=0)
        concat_gt_mask = np.concatenate((concat_gt_mask, gt_mask), axis=2)
        # Concatenate model "detection result"
        concat_r_rois = np.concatenate((concat_r_rois, r['rois']), axis=0)
        concat_r_masks = np.concatenate((concat_r_masks, r['masks']), axis=2)
        concat_r_class_ids = np.concatenate((concat_r_class_ids, r['class_ids']), axis=0)
        concat_r_scores = np.concatenate((concat_r_scores, r['scores']), axis=0)
    
    count1 += 1
    
    # - compute gt_tot and pred_tot - 
    gt, pred = utils.gt_pred_lists(gt_class_id, gt_bbox, r['class_ids'], r['rois'])
    gt_tot = np.append(gt_tot, gt)
    pred_tot = np.append(pred_tot, pred)
    
    if len(gt_class_id) == 0 and len(r['class_ids']) > 0:
        gt_tot_per_image = np.append(gt_tot_per_image, np.array(0))
        pred_tot_per_image = np.append(pred_tot_per_image, np.max(r['class_ids']))
        cls = np.max(r['class_ids'])
        if dataset.class_names[int(cls)] not in class_names:
            class_names.append(dataset.class_names[int(cls)])
            class_names_index.append(int(cls))
            
    elif len(gt_class_id) > 0 and len(r['class_ids']) == 0:
        gt_tot_per_image = np.append(gt_tot_per_image, np.max(gt_class_id))
        pred_tot_per_image = np.append(pred_tot_per_image, np.array(0))
        cls = np.max(gt_class_id)
        if dataset.class_names[int(cls)] not in class_names:
            class_names.append(dataset.class_names[0])
            class_names_index.append(0)
            
    elif len(r['class_ids']) == 0 and len(gt_class_id) == 0:
        gt_tot_per_image = np.append(gt_tot_per_image, np.array(0))
        pred_tot_per_image = np.append(pred_tot_per_image, np.array(0))
        if dataset.class_names[0] not in class_names:
            class_names.append(dataset.class_names[0])
            class_names_index.append(0)
            
    else:
        gt_tot_per_image = np.append(gt_tot_per_image, np.max(gt_class_id))
        pred_tot_per_image = np.append(pred_tot_per_image, np.max(r['class_ids']))
        cls = np.max(gt_class_id)
        if dataset.class_names[int(cls)] not in class_names:
            class_names.append(dataset.class_names[int(cls)])
            class_names_index.append(int(cls))
        
    color_list = []
    cls_index = []
    
    for m, cls_name in enumerate(r['class_ids']):
        # if cls_name == 3:
            # color_list.append(colors[2])
        # if cls_name == 2:
            # color_list.append(colors[1])
        # if cls_name == 1:
            # color_list.append(colors[0])
        cls_index.append(cls_name)
    
    dice = []
    for j in range(len(gt_class_id)):
        gt = gt_class_id[j].astype(np.int32) * np.array(gt_mask.astype(np.int32)[:,:,j])
        for k in range(len(cls_index)):
            seg = cls_index[k].astype(np.int32) * r["masks"].astype(np.int32)[:,:,k]
            score = np.sum(seg[gt==gt_class_id[j].astype(np.int32)])*2.0 / (np.sum(seg) + np.sum(gt))

            if score <= 1:
                dice.append(score)
    
    if np.mean(dice) <= 1:
        print('Dice similarity score is {}'.format(np.mean(dice)))
        mdice.append(np.mean(dice))
    
    image_display = visualize.display_instances(config.OUTPUT_DIR, info["id"], image, r['rois'], r['masks'], r['class_ids'], dataset.class_names, r['scores'], ax=ax) #, title="Predictions"
    plt.imshow(image_display)
    plt.savefig(config.OUTPUT_DIR+str(info["id"]))
    plt.close()
    
    AP, precisions, recalls, overlaps = utils.compute_ap(gt_bbox, gt_class_id, gt_mask, r['rois'], r['class_ids'], r['scores'], r['masks'])
    APs.append(AP)
    
    # visualize.plot_precision_recall(config.OUTPUT_DIR, info["id"][:-4], AP, precisions, recalls)
    # visualize.plot_overlaps(config.OUTPUT_DIR, info["id"][:-4], gt_class_id, r['class_ids'], r['scores'], overlaps, dataset.class_names)

    log("gt_class_id", gt_class_id)
    log("gt_bbox", gt_bbox)
    log("gt_mask", gt_mask)
"""

In [ ]:
"""
# ### Precision-Recall

print("mAP @ IoU=50: ", np.mean(APs))
print("mean Dice: ", np.mean(mdice))

# [change] Confusion matrix ===================================
y_test = np.array(gt_tot_per_image)
predictions = np.array(pred_tot_per_image)

confm = confusion_matrix(y_test, predictions)
df_cm = DataFrame(confm, index=[dataset.class_names], columns=[dataset.class_names])

sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, cmap='Blues', fmt='g') # font size

plt.title('Confusion matrix (per image)')
plt.savefig(config.OUTPUT_DIR+"Confusion_matrix.png")
#plt.show()
plt.close()

# [change] F1 scores ==========================================
fig, ax = plt.subplots(1, 1, figsize=(9, 6), tight_layout=True)
    
colorlist = colorlist = ['Gray', 'Green', 'Yellow', 'Red', 'Blue', 'Green', 'Yellow', 'Red', 'Blue'] #config.colorlist

F1_scores = []

for class_id in range(1, len(dataset.class_names)):
    mAP, precisions, recalls, _ =\
            utils.compute_ap_pre_class(concat_gt_bbox, concat_gt_class_id, concat_gt_mask, concat_r_rois, concat_r_class_ids, concat_r_scores, concat_r_masks, class_id)
    mAR, _ = utils.compute_recall(concat_r_rois, concat_gt_bbox, iou=0.5) 
    
    F1_scores = (2* (np.mean(precisions) * np.mean(recalls)))/(np.mean(precisions) + np.mean(recalls))
    
    ax.plot(recalls, precisions, color = colorlist[class_id], label=dataset.class_names[class_id] + ' mAP=%.2f, F1 score=%.2f' % (mAP, F1_scores))

ax.set_xlabel('Recall (Sensitivity, True positive rate)')
ax.set_ylabel('Precision (Positive predicted values)')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

ax.legend(loc="upper right", prop={'size': 12})

# [change] PR curve ===========================================
mmAP, mprecisions, mrecalls, overlaps =\
            utils.compute_ap(concat_gt_bbox, concat_gt_class_id, concat_gt_mask,
                     concat_r_rois, concat_r_class_ids, concat_r_scores, concat_r_masks)
mmAR, _ = utils.compute_recall(concat_r_rois, concat_gt_bbox, iou=0.5) 
F1_scores = (2* (np.mean(mprecisions) * np.mean(mrecalls)))/(np.mean(mprecisions) + np.mean(mrecalls))
ax.plot(mrecalls, mprecisions, color = 'Blue', label='Total mAP=%.2f, F1 score=%.2f, Mean dice similarity score=%.2f' % (mmAP, F1_scores, mdice))

plt.legend()
plt.savefig(config.OUTPUT_DIR+"PR_curve.png")
plt.close()
# =============================================================

"""